In [1]:
from typing import Any, Dict, List, Optional

from pydantic import BaseModel, root_validator

# Code cobbled together from various sources:
# https://github.com/samuelcolvin/pydantic/discussions/3091
# https://github.com/samuelcolvin/pydantic/issues/2177
# https://github.com/samuelcolvin/pydantic/discussions/3906

class InheritanceAware(BaseModel):

    t: str
    
    # used to register automatically all the submodels in `_types`.
    _subtypes_: Dict[str, type] = {}
    def __init_subclass__(cls):
        cls._subtypes_[cls.__name__] = cls
    
    @classmethod
    def __get_validators__(cls):
        yield cls._convert_to_real_type_

    @classmethod
    def _convert_to_real_type_(cls, data):
        if issubclass(type(data), cls): return data
        data_type = data.get("t")

        if data_type is None:
            raise ValueError("Missing 'type' attribute")

        sub = cls._subtypes_.get(data_type)

        if sub is None:
            raise TypeError(f"Unsupported sub-type: {data_type}")

        return sub(**data)
    
    @classmethod
    def parse_obj(cls, obj):
        return cls._convert_to_real_type_(obj)
    
    @root_validator(pre=True)
    def set_t(cls, values):
        values['t'] = cls.__name__
        return values



In [2]:

class Pet(InheritanceAware):
    name: str

class Cat(Pet):
    age: int

class Dog(Pet):
    name: str
    food: str

class Person(BaseModel):
    pets: List[Pet]

cat_obj = {'t': 'Cat', 'name': 'Garfield', 'age': 7}
dog_obj = {'t': 'Dog', 'name': 'Snoopy', 'food': 'sandwich'}

# Correctly instantiates subclasses:
print(Person(pets=[cat_obj, dog_obj]))


# This doesn't instantiate the subclass because it calls the constructor directly:
cat = Pet(**cat_obj)
print("Using pet constructor:", type(cat))

# Instead use parse_obj():
cat = Pet.parse_obj(cat_obj)
print("Using Pet.parse_obj:", type(cat))

# parse_raw() also works because it calls parse_obj internally:
cat = Pet.parse_raw('{"t": "Cat", "name": "Garfield", "age": 7}')
print("Using Pet.parse_raw:", type(cat))

# Correctly serializes type information as "t":
print("cat.json():", cat.json())

# For this to work, we needed to add the following to _convert_to_real_type_:
#   if issubclass(type(data), cls): return data
print("Instantiate with objects:", Person(pets = [cat]))


pets=[Cat(t='Cat', name='Garfield', age=7), Dog(t='Dog', name='Snoopy', food='sandwich')]
Using pet constructor: <class '__main__.Pet'>
Using Pet.parse_obj: <class '__main__.Cat'>
Using Pet.parse_raw: <class '__main__.Cat'>
cat.json(): {"t": "Cat", "name": "Garfield", "age": 7}
Instantiate with objects: pets=[Cat(t='Cat', name='Garfield', age=7)]


In [3]:
from typing import Union

from pydantic import BaseModel


class Foo(BaseModel):
    pass


class Bar(BaseModel):
    pass


class Model(BaseModel):
    x: Union[str, int]
    y: Union[Foo, Bar]

    class Config:
        smart_union = True


print(Model(x=1, y=Bar()))
#> x=1 y=Bar()


x=1 y=Bar()


In [4]:
# Some boilerplate code to make the notebook work
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
from neuronbridge import model
import json
from devtools import debug

In [6]:
with open("../test_data_v3/mcfo-line.json") as f:
    obj = json.load(f)
    debug(model.ImageLookup(**obj))
    
    
    
    

/tmp/ipykernel_837637/1005187541.py:3 <cell line: 1>
    model.ImageLookup(**obj): ImageLookup(
        results=[
            LMImage(
                id='2711777306946306059',
                libraryName='FlyLight Gen1 MCFO',
                publishedName='R77F05',
                alignmentSpace='JRC2018_Unisex_20x_HR',
                gender=<Gender.male: 'm'>,
                files=Files(
                    ColorDepthMip=(
                        'JRC2018_Unisex_20x_HR/FlyLight_Gen1_MCFO/R77F05-20190430_64_H1-GAL4-m-40x-brain-JRC2018_Unise'
                        'x_20x_HR-CDM_1.png'
                    ),
                    ColorDepthMipThumbnail=(
                        'JRC2018_Unisex_20x_HR/FlyLight_Gen1_MCFO/R77F05-20190430_64_H1-GAL4-m-40x-brain-JRC2018_Unise'
                        'x_20x_HR-CDM_1.jpg'
                    ),
                    ColorDepthMipInput=None,
                    ColorDepthMipMatch=None,
                    ColorDepthMipSkel=None,
              

In [7]:
with open("../test_data_v3/mcfo-line.json") as f:
    obj = json.load(f)
    debug(model.ImageLookup(**obj))
    

/tmp/ipykernel_837637/288594718.py:3 <cell line: 1>
    model.ImageLookup(**obj): ImageLookup(
        results=[
            LMImage(
                id='2711777306946306059',
                libraryName='FlyLight Gen1 MCFO',
                publishedName='R77F05',
                alignmentSpace='JRC2018_Unisex_20x_HR',
                gender=<Gender.male: 'm'>,
                files=Files(
                    ColorDepthMip=(
                        'JRC2018_Unisex_20x_HR/FlyLight_Gen1_MCFO/R77F05-20190430_64_H1-GAL4-m-40x-brain-JRC2018_Unise'
                        'x_20x_HR-CDM_1.png'
                    ),
                    ColorDepthMipThumbnail=(
                        'JRC2018_Unisex_20x_HR/FlyLight_Gen1_MCFO/R77F05-20190430_64_H1-GAL4-m-40x-brain-JRC2018_Unise'
                        'x_20x_HR-CDM_1.jpg'
                    ),
                    ColorDepthMipInput=None,
                    ColorDepthMipMatch=None,
                    ColorDepthMipSkel=None,
               